# Оптимизировать производство



Известно, что есть 2 матрицы "Орион - 800".

Если заказ для этой матрицы большой, то можно установить обе матрицы и выполнять его одновременно.

Поэтому сначала делать проверку, а не разделить ли большой заказ. Как делить? - Пока не знаю, поэтому ...

In [1]:
import numpy as np
import pandas as pd

from itertools import combinations

In [2]:
input_file = "input_data.xlsx"
output_file = "result.xlsx"

In [10]:
sheetname = "Orders"
df_orders = pd.read_excel(input_file, sheetname, nrows=None)
df_orders

,MatrixName,OrderSize
0,Орион - 600,734
1,Орион - 700,722
2,Орион - 800,1622
3,Весна - 600,218
4,Весна - 700,350
5,Весна - 800,820
6,Сафари,292
7,Сафари 2,352
8,Сатурн ПГ,42
9,Сатурн ПО,584


- NORDERS
- NLOT - кол-во мест для матриц пресса
- TCHANGE
- THEAT
- DIFF_TOT_TIME

In [4]:
NORDERS = df_orders.shape[0]
NLOT = 3

TCHANGE = 20
THEAT = 40

In [5]:
ORDERS = set(df_orders.MatrixName.values)
ORDERS

{'Весна  - 700', 'Весна - 600', 'Орион - 600', 'Орион - 700', 'Орион - 800'}

In [6]:
def convert_dataframe_to_dict(dataframe, key_columns, value_column):
    return (
        dataframe.loc[:, key_columns + [value_column]]
        .set_index(key_columns)
        .to_dict()[value_column]
    )

In [7]:
DURATION = convert_dataframe_to_dict(df_orders, ["MatrixName"], "OrderSize")
DURATION

{'Орион - 600': 734,
 'Орион - 700': 722,
 'Орион - 800': 1622,
 'Весна - 600': 218,
 'Весна  - 700': 350}

## Задача

In [9]:
for comb_orders in combinations(DURATION.keys(), 3):
    print(comb_orders, ORDERS.difference(comb_orders))

('Орион - 600', 'Орион - 700', 'Орион - 800') {'Весна  - 700', 'Весна - 600'}
('Орион - 600', 'Орион - 700', 'Весна - 600') {'Орион - 800', 'Весна  - 700'}
('Орион - 600', 'Орион - 700', 'Весна  - 700') {'Орион - 800', 'Весна - 600'}
('Орион - 600', 'Орион - 800', 'Весна - 600') {'Весна  - 700', 'Орион - 700'}
('Орион - 600', 'Орион - 800', 'Весна  - 700') {'Орион - 700', 'Весна - 600'}
('Орион - 600', 'Весна - 600', 'Весна  - 700') {'Орион - 800', 'Орион - 700'}
('Орион - 700', 'Орион - 800', 'Весна - 600') {'Орион - 600', 'Весна  - 700'}
('Орион - 700', 'Орион - 800', 'Весна  - 700') {'Орион - 600', 'Весна - 600'}
('Орион - 700', 'Весна - 600', 'Весна  - 700') {'Орион - 600', 'Орион - 800'}
('Орион - 800', 'Весна - 600', 'Весна  - 700') {'Орион - 600', 'Орион - 700'}


In [ ]:
def get_subdict(dictionary, keys):
    return {k:dictionary[k] for k in keys}

In [ ]:
def get_min_item(dictionary):
    return min(dictionary.items(), key=lambda x: x[1])

In [ ]:
def reduce_values(dictionary, keys, value):
    for k in keys:
        dictionary[k] -= value
    return dictionary

In [ ]:
def get_rest_orders(dictionary):
    num_empty_lots = 0
    rest_orders = []
    for k, v in dictionary.items():
        if v == 0:
            num_empty_lots += 1
        else:
            rest_orders.append(k)
    return rest_orders, num_empty_lots 

In [ ]:
tmin = 1e15
seqmin = None
for comb_orders in combinations(DURATION.keys(), 3):
    current = DURATION.copy()
    
    t0 = 0
    prod_sequence = [comb_orders]
    min_lot, min_order = get_min_item(get_subdict(current, comb_orders))
    
    t0 += min_order + TCHANGE
    current = reduce_values(current, comb_orders, min_order)
    
    rest_orders, num_empty_lots = get_rest_orders(current)
    
    print(current, rest_orders, num_empty_lots)

In [ ]:
get_min_item({1:0, 2: 0, 3:1})